# Example MQTT Subscriber

In [1]:
#!/usr/bin/env python
import json
import time
import paho.mqtt.client as mqtt
# For this program you have to install mqtt. See
# http://www.steves-internet-guide.com/into-mqtt-python-client/
#  You can:      pip install paho-mqtt

import sys
sys.path.append("../")
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import extend_stream

In [2]:
def on_connect(client, userdata, flags, rc):
    if rc==0: client.connected_flag=True
    else: print("Bad connection! Returned code = ", rc)

In [3]:
def start_callback_thread(procs, stream_name, topic, host, client_id=None):
    """
    This function starts a callback thread called by MQTT
    when a message arrives. The function puts the message
    into the stream called stream_name.

    Parameters
    ----------
    procs: list of process metadata obtained from
           get_processes_and_procs(multicore_specification)
           from IoTPy.concurrency.multicore
    stream_name: str
           The name of a stream which receives these MQTT
           messages.
    topic: str
           The topic to which this stream subscribes
    host: The ip address or 'localhost' of the host to which
          this stream subscribes.

    """
    def on_message(client, userdata, msg):
        data = json.loads(
            str(msg.payload.decode("utf-8","ignore")))
        if data == '_finished':
            # Received a '_finished' message indicating that the 
            # thread should terminate, and no further messages will
            # be arriving on this stream.
            terminate_stream(procs, stream_name)
            # Terminate the callback thread.
            sys.exit()
        else:
            extend_stream(procs, [data], stream_name)

    # Create the client
    client = mqtt.Client(client_id)
    client.connected_flag = False
    client.connect(host)
    client.on_connect = on_connect
    client.on_message = on_message
    client.loop_start()

    # Wait for the connection to succeed.
    N_MAX_TRIES = 10
    TIME_BETWEEN_TRIES = 0.5
    TIME_WAIT_FOR_PUBLISHER = 4

    # Wait for connection to succeed as indicated
    # by client.connected_flag
    n_tries = 0
    while not client.connected_flag:
        time.sleep(TIME_BETWEEN_TRIES)
        n_tries += 1
        if n_tries > N_MAX_TRIES:
            break

    if n_tries <= N_MAX_TRIES:
        # This client has connected successfully.
        # Subscribe to the topic
        client.subscribe(topic, qos=1)
        # Wait for the MQTT Publisher to start publishing.
        time.sleep(TIME_WAIT_FOR_PUBLISHER)
    else:
        # This client was unable to subscribe.
        # Log or print this information
        print ('Client unable to subscribe.')

In [4]:
from IoTPy.helper_functions.print_stream import print_stream
from IoTPy.concurrency.multicore import get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import extend_stream

In [5]:
def mqtt_subscriber_test():
    """
    This function shows how to use MQTT to receive a
    stream that was sent using MQTT_Publisher.

    The mqtt_receive_agent prints its single input stream.
    The function start_callback_thread starts a thread
    which receives JSON input (see 'body in callback) from
    MQTT and puts the data into a stream called
    'mqtt_receive_stream'.

    Note that start_callback_thread is a function that
    starts a thread; it is not a thread or a thread target.

    For efficiency, the publisher publishes segments of a
    stream as opposed to sending each single element separately.

    """
    # Step 0: Define agent functions, source threads 
    # and actuator threads (if any).

    # Step 0.0: Define agent functions.
    # This is merely a dummy agent that does nothing other than print.
    def mqtt_receive_agent(in_streams, out_streams):
        print_stream(in_streams[0], 'mqtt_receive_stream')

    # Step 0.1: Define source thread targets (if any).
    # qtt_callback_thread imported from MQTT_Subscriber
    # This thread is started when processes are started.

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs, additional arguments.
    # The mqtt_receive_stream data type is 'x' for arbitrary.
    # If the data type is int use 'i', and use 'f' for float.
    multicore_specification = [
        # Streams
        [('mqtt_receive_stream', 'x')],
        # Processes
        [{'name': 'mqtt_receive_process', 'agent': mqtt_receive_agent,
          'inputs':['mqtt_receive_stream'], 'sources': ['mqtt_receive_stream']}]]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # We skip these steps because we will call start_callback_thread
    # to start a thread.
    # Step 3: Create threads (if any)
    # The function start_callback_thread will start the callback thread.
    # Step 4: Specify which process each thread runs in.
    # mqtt_callback_thread runs in this (the calling) process.

    # Step 5: Start, join and terminate processes.
    # Start the mqtt_callback loop thread. The parameters are:
    # (0) procs, (1) stream name, (2) the pub/sub topic,
    # (3) the host.
    start_callback_thread(
        procs, 'mqtt_receive_stream', 'topic', 'localhost')
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

In [6]:
mqtt_subscriber_test()

ConnectionRefusedError: [Errno 61] Connection refused